In [ ]:
import csv

#Print the list of columns
with open('FerraraFiles/ferrara_airbreak_int_edti_ciclabili_2021.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    list_of_column_names = []
    for row in csv_reader:
        list_of_column_names.append(row)
        break

for i in list_of_column_names:
    print(i)

In [ ]:
#import osm2geojson
import geopandas as gpd

#All'interno del wfs inviatomi per mail ci sono i dati riguardanti le strade, però per ora faccio solo la traduzione dei tags

#Create the dataframes with the data of Ferrara, first of all the one about the ways, then that of the racks

dfWays = gpd.read_file("FerraraFiles/Ciclabili_2021.shp")
for col in dfWays.columns:
    print(col)

dfNodes = gpd.read_file("FerraraFiles/RastrelliereFerrara.shp")
for col in dfNodes.columns:
    print(col)

In [ ]:
dfWays.head()

In [ ]:
def rimuoviElementi(dataframe):
    dataframe.iloc[0:0]
    return dataframe

In [ ]:
#Extract only the name of the tags, the rest isn't needed now
dfWaysColumns = rimuoviElementi(dfWays)
dfWaysColumns.head()

In [ ]:
dfNodesColumns = rimuoviElementi(dfNodes)
dfNodesColumns.head()

In [ ]:
dfWaysColumns = dfWaysColumns.columns.tolist()
dfNodesColumns = dfNodesColumns.columns.tolist()

In [ ]:
#Create a csv file containing all the keys used by Ferrara
def creaCsvFileKeys(rowToWrite, path):
    f = open(path,"w")
    out=csv.writer(f,delimiter=",")
    out.writerow(rowToWrite)
    f.close()

#Function used to create all the csv of every single tag and key. It contains all the unique tags used by Ferrara
def creaCsvFileTagTraduzione(rowToWrite,columnToWrite,path):
    f = open(path,"w")
    out=csv.writer(f,delimiter=",")
    out.writerow(rowToWrite)
    for word in columnToWrite:
        if word=="None" or word is None:
            #print("None is the problem")
            out.writerow(["None"])
        else:
            out.writerow([word])
    f.close()

In [ ]:
#Create a csv file with the data of the ways and nodes. Later the translation will be added
creaCsvFileKeys(dfWaysColumns, "FerraraFiles/csvFiles/way_Ferrara_tags.csv")
creaCsvFileKeys(dfNodesColumns, "FerraraFiles/csvFiles/node_Ferrara_tags.csv")

In [ ]:
#Creo un csv per ogni elemento della colonna, così posso fare tutte le traduzioni dei valori che vengono inseriti(?)
#Per prima cosa cerco la chiave rappresentativa (per esempio segnaletic la chiave è traffic sign e road marking), successivamente per ogni risposta possibile inserisco il tag
#Questa era l'idea iniziale ma molti tag vengono tradotti con diversi elementi di OSM quindi faccio tabella che mostra le chiavi e i tag associati a ogni risposta possibile di quelli di Ferrara

In [ ]:
creaCsvFileTagTraduzione([" ","segnaletic"], dfWays['segnaletic'].unique(),"FerraraFiles/csvFiles/TabellaTraduzioneProva.csv")

for i in dfWaysColumns:
    print(i)
    creaCsvFileTagTraduzione([" ",i], dfWays[i].unique(),"FerraraFiles/csvFiles/wayTags/ferraraTags" + i + ".csv")

for i in dfNodesColumns:
    print(i)
    creaCsvFileTagTraduzione([" ", i], dfNodes[i].unique(),"FerraraFiles/csvFiles/nodeTags/ferraraTags" + i + ".csv")

<h1>Fase di Associazione Tags</h1>

- Una volta creati i csv per ogni dato di Ferrara (che si trovano nelle cartelle "FerraraFiles/csvFiles/nodeTags" e "FerraraFiles/csvFiles/wayTags"), devo riempire le tabelle con le traduzioni.

- Purtroppo questa fase è da compilare a mano in quanto è di traduzione. Ho inserito le traduzioni in file excel che possono essere facilmente convertiti in csv

- Carico i file convertiti in csv qui su python e effettuo le associazioni dei tag di Ferrara con quelli di OSM

- I file excel possono essere consultati nella cartella excelFiles.

Lo standard per leggere i csv e scrivere il file excel al fine di fare le associazioni è il seguente:
- La prima cella del file excel (cella 0,0) deve essere vuota. La seconda colonna prima riga (cella 0,1) deve contenere il nome della chiave del comune (es.: livello)
- Nome del File csv/excel deve avere all'interno la chiave presa in considerazione del comune/ente (per es.: ferraraTagsilllumnaz.csv ha come chiave il tag illuminaz)
- Nel file csv/excel la prima colonna rappresenta i valori che possono essere assegnati alla chiave del comune (per es.: si), le colonne successive mostrano la traduzione della chiave e del tag considerato(per es.: chiave=illuminaz, tag=si, traduzione=lighting=yes)
- Se non vi è una traduzione si può lasciare il campo excel vuoto, tuttavia Se non viene inserito nulla i campi senza traduzione verranno ignorati e quindi eliminati. Se si vogliono tenere almeno i tag senza una traduzione allora inserire 0 nel campo traduzione
- Se può essere tradotto in più modi, scrivere le traduzioni di colonna in colonna
- non andare a capo all'interno delle celle
- Se un tag può avere più valori è necessario separarli con uno slash ("/"), per es.: bicycle=yes/no/...
- Se un elemento deve essere tradotto tramite la composizione di più tag, separare questi con un +. Per es.: su marciapiede ----> highway=cycleway/footway/path + bicycle=designated + foot=designated + segregated=yes + sidewalk=yes/both/separate/right/left
- Il file va quindi letto di riga in riga per vedere le varie associazioni riguardanti i vari tag

<h1>Association Tags Phase</h1>

- Once all the csv files have been created (they can be found in the "FerraraFiles/csvFiles/nodeTags" and "FerraraFiles/csvFiles/wayTags" folders), I need to fill the tables with the translations.

- Sadly this phase has to be done by hand since it's a translation phase. I put the translation in the excel files and they can be easily converted in csv files.

- Load all the excel files converted in csv files here in python and make the associations of the tags of Ferrara with those of OSM

- The exxcel files can be consulted in the excelFiles folder

The standard used to read the csv files and used to write the excel files is described here:
- The first cell of the excel file (cell n. 0,0) has to be empty. The second column of the first row (cell 0,1) has to contain the name of the key of the public administration (for example.: level)
- The name of the csv/excel file has to have the name of the key taken into consideration (for example.: ferraraTagslevel.csv has the key level)
- In the excel/csv file the first column represents the values that can be assigned to the key of the public administration (for example.: yes), the next columns show the translation of the key or tag considered (for example.: key=lighting, tag=yes, translation=lighting=yes)
- If there's no translation then you can leave the excel field empty, byt if you do so then the field gets ignored and eliminated. If you want to keep the tags without a translation then you can put a 0 in the translation field
- If something can be translated in multiple ways then write the translation in subsequent columns
- Don't make new lines in the cells.
- If a tag can have multiple values then it is necessary to separate them with a slash "/", for example.: bicycle=yes/no/...
- If an element needs to be translated throught the composition of multiple tags, separate them with a +. For example.: on sidewalk ----> highway=cycleway/footway/path + bicycle=designated +foot=designated + segregated=yes + sidewalk=yes/both/separate/right/left
- The file needs to be read line by line in order to see the various association of the various tags.

In [ ]:
!pip install openpyxl

In [ ]:
#The excel files are loaded in the excelFiles folder
import pandas as pd
import os

excelFilesFolder = "FerraraFiles/excelFiles"
translatedCsvFiles = "FerraraFiles/translationFiles/"

#Convert the excel files into csv
for i in os.listdir(excelFilesFolder):
    f = os.path.join(excelFilesFolder, i)
    if(os.path.isfile(f)):
        print(f)
        #converti to csv
        read_file = pd.read_excel(f)
        read_file.to_csv(translatedCsvFiles + i.replace(".xlsx",".csv",1), index=None, header=True)

In [ ]:
import re

#Not used
def splitFunction(character,element):
    print(element)
    element = element.split(character);
    return element

#Not used
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

#check if the string contains any /
def checkIfContainsSlashes(element):
    c = "/"
    if c in element:
        #print("si contiene lo slash")
        return True
    else:
        #print("QUINDI QUESTO " + element + "NON CONTIENE SLASH?")
        return False

In [ ]:
#Recursive function that returns all the possible combination of tags written inside the csv
#For example.: If I have the element bicycle=yes/no + highway=cycleway/footway then it returns the list [[bicycle=yes + highway=cycleway],[bicycle=no + highway=cycleway],[.......]]
def recursiveChangeCsvStructure(element):
    #print("elemento preso in considerazione: {}" .format(element))
    my_list=[]
    my_second_list = []
    #c="/"
    allTagsAndKeys="" #For Ex.: (highway=cycleway/footway) rappresenta key e tags
    singleKeyTag = ""
    #key=""
    tags = ""
    #word_between = ""
    new_element = ""
    replacedWith =""
    #addedColumns = 0
    #If the string doesn't have "/" then it  doesn't need to be modified
    if checkIfContainsSlashes(element)==False:
        #print("l'element {} Non contiene slash" .format(element))
        my_list.append(element)
        return my_list
    #print("l'element {} CONTIENE lo slash" .format(element))
    #word_between = re.search('=(.*?)/', element)
    allTagsAndKeys = element.split('+')
    for i in range(len(allTagsAndKeys)):
        allTagsAndKeys[i] = allTagsAndKeys[i].replace(" ","")
    #Now I have to take the single tags and key, then I have to create new fields for every single one of them, leaving the rest as it is
    for i in range(len(allTagsAndKeys)):
        singleKeyTag = allTagsAndKeys[i].split("=")
        for i in range(len(singleKeyTag)):
            if i==0:
                key=singleKeyTag[i]
            else:
                tags = singleKeyTag[i].split("/")
                #The first element of this tags doesn't need to be considered because I'll leave it where it is
                for j in range(len(tags)):
                    if j==0:
                        pass
                    else:
                        #singleKeyTag[i]
                        new_element = element.replace(singleKeyTag[i],tags[j])
                        #print("IL NUOVO ELEMENTO: " + new_element)
                        replacedWith = element.replace("/"+tags[j],"")
                        #print("REPLACED WITH E': " + replacedWith) 
                        my_list.append(new_element)
                        my_list.append(replacedWith)
                        my_list = recursiveChangeCsvStructure(new_element)
                        my_second_list = recursiveChangeCsvStructure(replacedWith)
                        #my_list = recursiveChangeCsvStructure(my_list)
                        #print("la lista my_list è {}" .format(my_list))
                        #print("la lista second_list è {}" .format(my_second_list))
                        my_list = my_list + my_second_list
                        #print("la mia lista finale è")
                        #print(my_list)
                        return my_list

In [ ]:
import csv
import pandas as pd

#Function needed to create the csv files with all the possible tags and combinations
def createFinalCsv(filePath, finalFolder):
    number_of_row=0
    key_list = []
    public_admin_tag = ""
    list_of_rows=[]
    single_row = []
    list_to_write = []

    with open(filePath) as csv_file:
        csv_reader = csv.reader(csv_file,delimiter = ",")
        list_of_column_names = []
        for row in csv_reader:
            if number_of_row==0:
                #Remove the first element becase it's an empty cell. Otherwise it writes unnamed
                row.pop(0)
                #Reinsert the empty cell
                list_of_rows.append(row)
                key_list.append("")
                #The first row contains only the key taken in consideration
                pass
            else:
                #row[0] represents the tag of the public administration
                column=0 #need to take track of the number of columns
                for i in row:                        
                    if(column==0):
                        public_admin_tag = row[column]
                        #print("Tag del comune: " + row[column])
                        column = column + 1
                    else:
                        if row[column] != "" and row[column]!=None:
                            #print("Traduzione Per il Tag " + public_admin_tag+ ": " + row[column])
                            key_list.append(public_admin_tag)
                            single_row = recursiveChangeCsvStructure(row[column])
                            #print("aggiungo alla lista di lista la single_row con valore {}" .format(single_row))
                            list_of_rows.append(single_row)    
                        column = column +1
            number_of_row = number_of_row+1
        for i in range(len(key_list)):
            new_list = []
            new_list = list(list_of_rows[i])
            new_list.insert(0,key_list[i])
            list_to_write.append(new_list)
        with open(finalFolder, "w") as f:
            writer = csv.writer(f)
            for i in list_to_write:
                #print("Writing.... {}" .format(i))
                writer.writerow(i)
        #Fine

import pandas as pd
import os

csvFilesFolder = "FerraraFiles/translationFiles"
finalFolder = "FerraraFiles/Finalcsv/"

for i in os.listdir(csvFilesFolder):
    f = os.path.join(csvFilesFolder, i)
    if(os.path.isfile(f)):
        #print(f)
        finalFile = finalFolder + "final_" + i
        #print(finalFile)
        #print(f)
        createFinalCsv(f, finalFile)

In [ ]:
#TODO remove these comments...
#Forse conviene fare un megacsv dopo aver creato tutti i singoli che contiene tutte le possibili traduzioni. 
#Cioè inserisco la coppia chiave e valore del comune, tipo "su marciapiede = no" e nelle colonne dopo tutte le traduzioni
#Faccio così un unico file che tiene traccia di tutte le combinazioni possibili di quello di ferrara
#Così non devo neanche stare a cercare i file dal nome corretto.
import pandas as pd
import os


def create_new_rows(filePath):
    number_of_row=0
    column = 0
    key=""
    new_row = []
    new_first_column = ""
    list_of_rows = []
    with open(filePath) as csv_file:
        csv_reader = csv.reader(csv_file,delimiter = ",")
        list_of_column_names = []
        for row in csv_reader:
            if number_of_row==0:
                #Remove the first element because it is an empty cell
                key = row[1]
                number_of_row+=1
            else:
                #print(row)
                new_row = row
                new_first_column = key + "=" + row[0]
                new_row[0] = new_first_column
                list_of_rows.append(new_row)
                #print(list_of_rows)
                number_of_row+=1
                #print(new_first_column)
                #new_row

                #for i in row:
        return list_of_rows 
        #Fine
list_of_all_csv_rows = [] #will be a list of list where every element represents a row of the csv file
finalFolder = "FerraraFiles/Finalcsv/"
single_row = []

for i in os.listdir(csvFilesFolder):
    f = os.path.join(csvFilesFolder, i)
    if(os.path.isfile(f)):
        finalFile = finalFolder + "final_" + i
        #print(finalFile)
        single_row = create_new_rows(finalFile)
    list_of_all_csv_rows.append(single_row)
    #Create the csv file
"""file="FerraraFiles/Finalcsv/final_ferraraTagslivello.csv"
single_row = create_new_rows(file)
print("ECCOCI QUA {}" .format(single_row))
list_of_all_csv_rows.append(single_row)"""



In [ ]:
#print(list_of_all_csv_rows)
fileName = "FerraraFiles/Finalcsv/translationFileEverything.csv"
firstRow = ["PUBLICADMINISTRATIONTAG","TRANSLATION"]

#Write everything in a single file
with open(fileName, "w") as f:
    writer = csv.writer(f,delimiter=",")
    writer.writerow(firstRow)
    for i in list_of_all_csv_rows:
        for j in i:
            publicAdministrationTag = j[0]
            for position in range(len(j)):
                if(position)==0:
                    pass
                else:
                    list_to_write = []
                    list_to_write.append(publicAdministrationTag)
                    list_to_write.append(j[position])
                    writer.writerow(list_to_write)



In [1]:
#From the first row we keep only the key (since the rest is unnamed or empty). This key can be found in the column in position 1

#Function that returns the translation of the searched tags. If it doesn't find a translation it returns an empty list
#osmOrAdministration is necessary to know if we want to translate from OSM to the public administration or the other way around
#tags represents the tags tht we want to search the translation for (for example.: "highway=cycleway")
#fileName represents the path of the csv file in which there's the table with the associations OSM---->Public administration
def searchTranslation(osmOrAdministration, tags, fileName):
    index = 0
    if(osmOrAdministration=="OSM"):
        index = 0
    else:
        index = 1
    translation_list = []
    with open(fileName) as csv_file:
        csv_reader = csv.reader(csv_file,delimiter=",")
        for row in csv_reader:
            if(row[1] in tags):
                #The sent element contains also the tags in the csv, so it respects the parameters needed for the key
                translation = row[index]
                translation_list.append(translation)
            #elif so I avoid that it prints 2 times if the tags are the same
            elif tags in row:
                translation = row[index]
                translation_list.append(translation)
    return translation_list

#With all merged files
"""chiaveEsempio = "tipo_fondo"
tagEsempio = "asfalto"
word_to_search = chiaveEsempio + "=" + tagEsempio"""

#EXAMPLES
fileName = "FerraraFiles/Finalcsv/translationFileEverything.csv"

#EXAMPLES FROM OSM TO PUBLIC ADMINISTRATION
"""osmValue = "surface=asphalt"
traduzioni = searchTranslation("OSM", osmValue, fileName)
if(len(traduzioni)==0):
    print("Not Found")
for i in range(len(traduzioni)):
    print("La traduzione per il valore {} è {}".format(osmValue,traduzioni[i]))
"""

#EXAMPLE FROM PUBLIC ADMINISTRATION TO OSM
"""osmValue = "livello=su marciapiede"
traduzioni = searchTranslation("public", osmValue, fileName)
if(len(traduzioni)==0):
    print("Not Found")
for i in range(len(traduzioni)):
    print("La traduzione per il valore {} è {}".format(osmValue,traduzioni[i]))
"""

#EXAMPLE WITH MULTIPLE TAGS FROM OSM TO PUBLIC ADMINISTRATION
osmValue = "highway=cycleway + bicycle=designated + foot=designated + segregated=no + sidewalk=right"
traduzioni = searchTranslation("OSM", osmValue, fileName)
if(len(traduzioni)==0):
    print("Not Found")
for i in range(len(traduzioni)):
    print("La traduzione per il valore {} è {}".format(osmValue,traduzioni[i]))



FileNotFoundError: [Errno 2] No such file or directory: 'FerraraFiles/Finalcsv/translationFileEverything.csv'

In [ ]:
"""
QUESTO PUò FUNZIONARE DA UNA PARTE CON I FILE SINGOLI. NON SERVE CHE SIA CHIAVE=TAG, MI BASTANO SEMPLICEMENTE I DUE VALORI NON UNITI
key=list_of_column_names[0][1]
print(key)
#Ora che ho la chiave posso vedere se riesco a tradurre un tag. Mettiamo caso che mi arrivi tipo_fondo = asfalto
chiaveEsempio = "tipo_fondo"
tagEsempio = "asfalto"
translated_list = []
#Ora devo tradurlo verificando nel file
#fileName = "FerraraFiles/Finalcsv/final_ferraraTags" + chiaveEsempio + ".csv" per cercarlo proprio nel suo file
with open(fileName) as csv_file:
    csv_reader = csv.reader(csv_file,delimiter =",")
    for row in csv_reader:
        if tagEsempio in row:
            print("found, the row is {}" .format(row))
            translated_list = row
            translated_list.pop(0)
            print(translated_list)
print("la coppia chiave: {} e tag {} hanno come traduzioni: {}".format(chiaveEsempio,tagEsempio,translated_list))
"""


In [ ]:
#PER DIRE STRADA A DOPPIO SENSO è ONEWAY=NO
#DEVO CAPIRE COME DIRE SEGNALETICA ORIZZONTALE O VERTICALE IN INGLESE    
#https://help.openstreetmap.org/questions/6320/how-do-i-map-a-sidewalkpavement-with-shared-use-for-bicycle-and-pedestrians